In [16]:
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
import re
import pandas as pd

In [2]:
# Funcion para convertir el contenido de un pdf a txt
def getPDFFileContentToTXT(pdfFile):
    myPDFFile = PyPDF2.PdfFileReader(pdfFile)
    
#Creamos un archivo txt con la informacion del pdf 
    with open('pdfContenido.txt', 'w') as pdf_output:
        for page in range (myPDFFile.getNumPages()):
            data = myPDFFile.getPage(page).extractText()
            pdf_output.write(data)

# Leemos la infromacion que acabamos de escribir e quitamos los espacios de lineas
    with open('pdfContenido.txt', 'r') as myPDFContent:
        return myPDFContent.read().replace('\n',' ')

## Programa para analizar y calificar CV´s por Andres Saldaña

### Requerimientos de Vacantes

In [17]:
df = pd.read_csv('Requisitos_Vacantes.csv')

In [23]:
df = df.drop("Otros",axis = 1)

In [25]:
df

,Igualador,Facturacion
0,Igualación de colores,Facturación de clientes
1,Captura de bitácoras,Cobro de ventas mostrador
2,Trabajo en equipo,Elaboración de corte diario
3,Pintura automotriz,Elaboración de Pedidos
4,NaN,Encargado de facturación


### Convertir CV a TXT

In [3]:
pdfFileContent = getPDFFileContentToTXT('CV.pdf')

In [4]:
pdfFileContent

'Luis Hernández ramosTuxtla Gutiérrez, Chis.luisramos170300@gmail.com9614486352Experiencia laboralIgualador automotriz y encargado de sucursalComercializadora de pinturas y complementos s.a de c.v  -  Tuxtla Gutiérrez, Chis. abril 2019 - enero 2022 Igualador de pintura automotriz en la línea ppg Esmaltes  acrílicos,vendedor  de  mostrador  con  buena  presentación,puntualidad  con responsabilidad,buena administración de una sucursal ,con todas las ganas de trabajar y demostrar un buen trabajo con la visión de escálalar Educación y formación Bachillerato trunco o en curso en Administración de recursos humanos Cetis 138  -  Tuxtla Gutiérrez, Chis. Habilidades, conocimientos e idiomas  •    manejo de computadora    •    microsoft office    •    manejo de excel    •    manejo de personal    •    microsoft office  '

### Separamos las palabras pegadas TXT

In [5]:
count = 1

for letter in range(len(pdfFileContent)): #funcion para limpiar y corregir errores de transformacion a txt
    
    count +=1
    if count == len(pdfFileContent):
        break
    
    l1 = pdfFileContent[letter]
    l2 = pdfFileContent[letter + 1]
    l3 = pdfFileContent[letter + 2]
    
    if l1.islower() == True and l2.isupper() == True and l3.islower() == True: #Errores de \n
        print("Errores por separar (letras): " + l1,l2,l3)
        pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)
        
    if l1.isdigit() == True and l2.isupper() == True and l3.islower() == True: #Numeros pegados a letras mayusculas 
        print("Errores por separar (numero): " + l1,l2,l3)
        pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)

Errores por separar (letras): s T u
Errores por separar (numero): 2 E x
Errores por separar (letras): l I g
Errores por separar (letras): l C o


### Tokenize y removemos caracteres innecesarios (Tokens 1)

In [11]:
Tokens_1 = word_tokenize(pdfFileContent)
#Tokens_pdfFileContent

In [12]:
punctuation = re.compile(r'[-.?!,:;()•@[0-9]') #lista de caracteres que removeremos

Tokens_post_punctuation = []
for words in Tokens_1:
   word = punctuation.sub("",words)
   if len(word)>0:
        Tokens_post_punctuation.append(word)

In [33]:
Tokens_post_punctuation[0:6] #tokens con puras palabras sin caracteres

['Luis', 'Hernández', 'ramos', 'Tuxtla', 'Gutiérrez', 'Chisluisramos']

### Lemmetize y remove stop words de el CV y los Requerimientos (Tokens 2)

### Buscar Requerimientos en CV | Bag Of Words

### Dar una Calificacion Basada en Los Requerimientos 